# AEWS Python Notebook 07c: generating L8 WQ data

**Author**: Eric Lehmann, CSIRO Data61  
**Date**: 25 May 2016 (slight update: July 15, 2016)

**Note**: The Python code below is "rudimentary" etc. etc. Priority is here given to code interpretability rather than execution efficiency.

**Note**: this notebook should be accessible and viewable at [https://github.com/eric542/agdc_v2/tree/master/notebooks](https://github.com/eric542/agdc_v2/tree/master/notebooks).

## Summary

In this notebook, we introduce / review the following concepts:

* loading PQ-masked Landsat / WQ data for an area and time range of interest, using AE/EE
* "removing" duplicated dates from the dataset
* reproject WQ data to geodetic
* loading WOFS data for the same area
* creating and applying the WOFS/water masks to the WQ data
* doing the above for multiple dates and saving each result to .png file
* saving associated ancillary data in .json format

This version `07c` is an updated duplicate of `07b` applied to L8 dataset.

## Preliminaries

This notebook summarises the processes developed / tested in the previous notebooks in this series. The aim is to demonstrate the automated production of WQ maps for a given lake / waterbody of interest, ready to be ingested in the AEWS visualisation interface module.

The code below generates output files for *all* dates available on the NCI for the region of interest, or *all* dates in the selected date range. Ultimately, this process will only be executed once, with follow-up executions only updating the "database" with the latest imagery, if new data is available.

The processes that are still missing from a "full-blown" AEWS implementation are:

* automatically reading the lat/lon extents of the waterbodies of interest from .shp file
* checking whether new data is available and needs to be processed (rather than reprocessing the entire time series)
* application of SWIR "glint" filter to the data
* comparison of SWIR filter against WOFS for further "low water" flagging

Also, further possible improvements are:

* use NDexpression language to optimise some steps, e.g. water masking of time series in one operation
* use updated AE/EE, e.g. to reproject data with `execute_plan()`

This (Jupyter) notebook was written for use on the NCI's VDI system, with the following pre-loaded module:

```
 $ module use /g/data/v10/public/modules/modulefiles --append
 $ module load agdc-py2-prod 
```

**NOTE**: the specific module loaded here (`agdc-py2-prod`) is different from the module loaded in earlier notebooks (`agdc-py2-dev`)! While the earlier module contained only Landsat 5 data, the `agdc-py2-prod` module links to a (different) AGDC database containing the following NBART/NBAR/PQA datasets:

* Landsat 8: 2013
* Landsat 7: 2013
* Landsat 5: 2006/2013

It is unclear whether the API functions in these 2 modules are identical or represent different versions.

In [1]:
%matplotlib inline
import matplotlib.image as img
import matplotlib.pyplot as plt
import numpy as np
from osgeo import gdal, osr, ogr
import math
import json
import os

from pprint import pprint
from __future__ import print_function

import datacube.api   # import the AGDC v2 API, with pre-loaded 'agds-py2-dev' (or similar) NCI module

from datetime import datetime
from datacube.analytics.analytics_engine import AnalyticsEngine
from datacube.execution.execution_engine import ExecutionEngine

from scipy import ndimage
from scipy.spatial.distance import pdist

def expand_mask(mask_arr, npix):
    # Uses the True/False (masked/non-masked) values in the array 'mask_arr' and 
    # expands the True values spatially by 'npix' pixels. The value 'npix' can be
    # non-integer, i.e. the mask can be expanded by any spatial distance.
    nmid = np.floor(npix)
    nmax = int( nmid*2 + 1 )
    struc = np.zeros((nmax, nmax), dtype='bool')
    for ii in range(nmax):   # create desired binary structure for morphological operation
        for jj in range(ii,nmax):
            if pdist( [[nmid,nmid], [ii,jj]] ) <= npix:
                struc[ii,jj] = True
                struc[jj,ii] = True
    return ndimage.binary_dilation(mask_arr, structure=struc)

In [2]:
# Investigation of available L8 dataset:

# dc = datacube.api.API()
# print( dc.list_field_values('product') )  # [u'pqa', u'ortho', u'DEM', u'satellite_telemetry_data', u'nbart', u'nbar']
# print( dc.list_field_values('platform') ) # [u'LANDSAT_8', u'LANDSAT_5', u'SRTM', u'LANDSAT_7']

In [3]:
# query = {   # whole of Australia!!
#     'product': 'nbar',
#     'platform': 'LANDSAT_8',
# }
# desc = dc.get_descriptor(query, include_storage_units=False)
# pprint(desc)

#{u'ls8_nbar_albers': {'dimensions': [u'time', u'y', u'x'],
#                      'irregular_indices': {u'time': array(['2013-03-19T11:30:08.884702000+1100',
#       '2013-03-19T11:30:32.710102000+1100',
#       '2013-03-19T11:30:56.540705000+1100', ...,
#       '2013-12-31T12:29:23.398040000+1100',
#       '2013-12-31T12:29:47.347583000+1100',
#       '2013-12-31T12:30:11.303643000+1100'], dtype='datetime64[ns]')},
#                      'result_max': (numpy.datetime64('2013-12-31T12:30:11.303643000+1100'),
#                                     -900012.5,
#                                     2399987.5),
#                      'result_min': (numpy.datetime64('2013-03-19T11:30:08.884702000+1100'),
#                                     -4999987.5,
#                                     -1999987.5),
#                      'result_shape': (6030, 164000, 176000),
#                      'variables': {u'band_1': {'datatype_name': dtype('int16'),
#                                                'nodata_value': -999},
#                                    u'band_2': {'datatype_name': dtype('int16'),
#                                                'nodata_value': -999},
#                                    u'band_3': {'datatype_name': dtype('int16'),
#                                                'nodata_value': -999},
#                                    u'band_4': {'datatype_name': dtype('int16'),
#                                                'nodata_value': -999},
#                                    u'band_5': {'datatype_name': dtype('int16'),
#                                                'nodata_value': -999},
#                                    u'band_6': {'datatype_name': dtype('int16'),
#                                                'nodata_value': -999},
#                                    u'band_7': {'datatype_name': dtype('int16'),
#                                                'nodata_value': -999}}}}

In [4]:
# query = { 'product':   'pqa',
#           'platform':  'LANDSAT_8',
#           'dimensions': { 'x':    {'range': (149.06, 149.17)},
#                           'y':    {'range': (-35.33, -35.27)},
#                           'time': {'range': ((2013, 1, 1), (2014, 1, 1))} } }
# desc = dc.get_descriptor(query, include_storage_units=False)
# pprint(desc)

#{u'ls8_pq_albers': {'dimensions': [u'time', u'y', u'x'],
#                    'irregular_indices': {u'time': array(['2013-04-12T09:46:35.385577000+1000',
#       '2013-04-19T09:52:06.192160000+1000',
#        [...]
#       '2013-12-31T10:51:42.297674000+1100',
#       '2013-12-31T10:52:06.251744000+1100'], dtype='datetime64[ns]')},
#                    'result_max': (numpy.datetime64('2013-12-31T10:52:06.251744000+1100'),
#                                   -3955437.5,
#                                   1554312.5),
#                    'result_min': (numpy.datetime64('2013-04-12T09:46:35.385577000+1000'),
#                                   -3963362.5,
#                                   1543537.5),
#                    'result_shape': (68, 318, 432),
#                    'variables': {u'pixelquality': {'datatype_name': dtype('int16'),
#                                                    'nodata_value': None}}}}

## User inputs

Here are the input parameters needed from the user:  

In [5]:
dimensions = { 'longitude': {'range': (149.06, 149.17)},   # selected region
               'latitude':  {'range': (-35.33, -35.27)},
               'time':      {'range': ((2013, 1, 1), (2014, 1, 1))} }

lake_str = "Lake_Burley_Griffin"        # (string) name of selected location / region
lake_dispname = "Lake Burley Griffin"   # (display) name of selected location / region

water_mask_thr = 90.0     # water percentage level to use to create water mask from WOFS
water_mask_buffer = 1.5   # buffer distance in pixels to expand the land mask by, i.e. "shrihnk" water mask
min_valid_pix_thr = 10.0  # minimum percentage of valid water pixels necessary to save image to file

save_basedir = '/g/data/jr4/vis_data_v1.1/'   # where the data is saved

WQ_type = "WQ_(B2+B3)/2"  # Type of WQ info generated by this code

## Loading / creating PQ-masked WQ data

**Important**: due to the availability of the Coastal Aerosol band in LS8 (not available in LS5,7), the "usual" Landsat bands are offset by one. For instance, the "usual" (green) band nr. 2 in LS5,7 actually corresponds to 'band_3' in LS8.

In [6]:
ae = AnalyticsEngine()
ee = ExecutionEngine()

aeB2 = ae.create_array(('LANDSAT_8', 'nbar'), ['band_3'], dimensions, 'aeB2')
aeB3 = ae.create_array(('LANDSAT_8', 'nbar'), ['band_4'], dimensions, 'aeB3')
aePQ = ae.create_array(('LANDSAT_8', 'pqa'), ['pixelquality'], dimensions, 'aePQ')
aeWQ = ae.apply_expression([aeB2, aeB3], '((array1 + array2) * 0.5)', 'aeWQ')
aeWQ_PQmsk = ae.apply_expression([aeWQ, aePQ], 'array1{array2}', 'aeWQ_PQmsk')
ee.execute_plan(ae.plan)

INFO:datacube.analytics.analytics_engine:Initialise Analytics Module.
INFO:datacube.execution.execution_engine:Initialise Execution Module.


In [7]:
# Extracting Y/M/D dates:
LS_dates = ee.cache['aeWQ_PQmsk']['array_indices']['time']
n_dates = len( LS_dates )
strLS_dates = np.zeros(n_dates).astype('str')
for ii in range(n_dates):
    strLS_dates[ii] = str( LS_dates[ii] )[:10]

In [8]:
# Extracting PQ-masked WQ arrays:
WQ_TS_xarr = ee.cache['aeWQ_PQmsk']['array_result']['aeWQ_PQmsk']
WQ_TS_arr = WQ_TS_xarr.values   # Time series dataset of WQ arrays

In [9]:
# "Removing" duplicated dates:
rem_ind = np.zeros(n_dates).astype('bool')   # keep track of which duplicated dates to remove

for ind in range(1,n_dates):
    dup_ind = np.where( strLS_dates[:ind]==strLS_dates[ind] )[0]   # check for duplicated date up to current index
    if len( dup_ind )!=0:   # found (at least) one duplicate
        dup_ind = dup_ind[0]   # only use the first index if multiple dates returned
        rem_ind[ind] = True    # remove current date index
        
        ind_n_nans = np.sum( np.isnan( WQ_TS_arr[ind] ) )   # nr of NaN pixels in each image
        dup_n_nans = np.sum( np.isnan( WQ_TS_arr[dup_ind] ) )
        
        if ind_n_nans==0:   # current data has no NaN's, use it instead of duplicate date (copy it to lowest index)
            WQ_TS_arr[dup_ind] = WQ_TS_arr[ind]
        elif dup_n_nans!=0:   # if duplicate date has no NaN's: do nothing (use it instead of current date)
            if dup_n_nans<ind_n_nans:   # duplicate date has less NaN's: fill it in with current data
                tmp = np.where( np.isnan(WQ_TS_arr[dup_ind]) )
                WQ_TS_arr[dup_ind][tmp] = WQ_TS_arr[ind][tmp]
            else:   # dup_n_nans>=ind_n_nans -- duplicate date has more NaN's: use it to fill in current data
                tmp = np.where( np.isnan(WQ_TS_arr[ind]) )
                WQ_TS_arr[ind][tmp] = WQ_TS_arr[dup_ind][tmp]
                WQ_TS_arr[dup_ind] = WQ_TS_arr[ind]   # save results to lowest date index

WQ_TS_arr = WQ_TS_arr[ ~rem_ind ]
strLS_dates = strLS_dates[ ~rem_ind ]

## Loading WOFS data and creating water mask

In [10]:
# This will create issues if regions spans multiple tiles...
mean_lon = np.mean( dimensions['longitude']['range'] )
mean_lat = np.mean( dimensions['latitude']['range'] )
base_folder = '/g/data2/fk4/wofs/current/pyramids/WaterSummary/0/'   # where geoTiff WOFS data is located
WOFS_fname = base_folder + 'percentWater_{:3.0f}_{:04.0f}.tiff'.format( mean_lon, math.floor(mean_lat) )

In [11]:
WOFS_dataset = gdal.Open(WOFS_fname)
WOFS_array = WOFS_dataset.ReadAsArray()
WOFS_geotx = list( WOFS_dataset.GetGeoTransform() )
WOFS_proj = WOFS_dataset.GetProjection()   # geodetic lat/lon
WOFS_srs = osr.SpatialReference( wkt=WOFS_proj )

WOFS_lonvec = np.arange(WOFS_array.shape[0]) * WOFS_geotx[1] + WOFS_geotx[0]
WOFS_latvec = np.arange(WOFS_array.shape[1]) * WOFS_geotx[5] + WOFS_geotx[3]

In [12]:
# Extract WOFS over region of interest:
yind_min = np.where( WOFS_lonvec>=dimensions['longitude']['range'][0] )[0][0]
yind_max = np.where( WOFS_lonvec<=dimensions['longitude']['range'][1] )[0][-1]
WOFS_lonvec = WOFS_lonvec[yind_min:yind_max+1]

xind_min = np.where( WOFS_latvec>=dimensions['latitude']['range'][1] )[0][-1]
xind_max= np.where( WOFS_latvec<=dimensions['latitude']['range'][0] )[0][0]
WOFS_latvec = WOFS_latvec[xind_min:xind_max+1]

WOFS_array = WOFS_array[xind_min:xind_max+1, yind_min:yind_max+1]
WOFS_geotx[0] = WOFS_geotx[0] + yind_min*WOFS_geotx[1]
WOFS_geotx[3] = WOFS_geotx[3] + xind_min*WOFS_geotx[5]

In [13]:
# generating the water / land mask:
land_mask = ( WOFS_array<=water_mask_thr )
land_mask = expand_mask( land_mask, water_mask_buffer )

## Reprojecting WQ data, water masking, and saving to file


In [14]:
# Get info about the Landsat data...

# The following should be read from the data directly but this info does not seem 
# to be available/extractable from AE/EE's outputs... (?)
dc = datacube.api.API()
query = { 'product': 'nbar',
          'platform': 'LANDSAT_8',
          'dimensions': dimensions }
LSdata = dc.get_data( query )

LS_proj_str = LSdata['coordinate_reference_systems'][1]['reference_system_definition']
LS_proj = osr.SpatialReference( wkt=LS_proj_str )

# Things that we can infer from the AE/EE results:
LSxvec = ee.cache['aeWQ_PQmsk']['array_result']['aeWQ_PQmsk']['x'].values
LSyvec = ee.cache['aeWQ_PQmsk']['array_result']['aeWQ_PQmsk']['y'].values
LSpix_size = np.mean(LSxvec[1:] - LSxvec[:-1])
LS_geotx = ( LSxvec[0], LSpix_size, 0, LSyvec[0], 0, -LSpix_size )

In [15]:
# WQ gdal dataset
WQ_dx, WQ_dy = WQ_TS_arr[0].shape
gdal_WQ_data = gdal.GetDriverByName( 'MEM' ).Create('', WQ_dy, WQ_dx, 1, gdal.GDT_Float32)
gdal_WQ_data.SetGeoTransform( LS_geotx )
gdal_WQ_data.SetProjection( LS_proj.ExportToWkt() );

In [16]:
# Reprojection variables
(WOFS_dx, WOFS_dy) = WOFS_array.shape
gdalgeo_WQ_data = gdal.GetDriverByName( 'MEM' ).Create('', WOFS_dy, WOFS_dx, 1, gdal.GDT_Float32)
gdalgeo_WQ_data.SetGeoTransform( WOFS_geotx )
gdalgeo_WQ_data.SetProjection( WOFS_proj );

In [17]:
# initialise contents.json data
contents_json_list = list()

In [18]:
# Create directory if necessary:
save_dir = save_basedir + lake_str + "/"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

Looping over all dates...

In [19]:
cur_datetime = datetime.now().strftime('%Y/%m/%d %H:%M:%S')

for dateind in range( len(strLS_dates) ):
    # "Save" WQ data as gdal dataset:
    gdal_WQ_data.GetRasterBand(1).WriteArray( WQ_TS_arr[dateind], 0, 0);
    
    # Reprojecting the WQ gdal dataset to match the WOFS coordinates:
    gdalgeo_WQ_data.GetRasterBand(1).WriteArray( np.ones((WOFS_dx,WOFS_dy))*np.nan, 0, 0)
    res = gdal.ReprojectImage( gdal_WQ_data, gdalgeo_WQ_data, 
                               LS_proj.ExportToWkt(), WOFS_proj, 
                               gdal.GRA_Bilinear )  # gdal.GRA_NearestNeighbour by default
    
    WQ_array = gdalgeo_WQ_data.ReadAsArray()   # reprojected result
    WQ_array[land_mask] = np.nan   # masking non-water pixels out
    
    # Saving image to file, only if enough valid data...
    n_water_pix = np.sum( ~land_mask )
    n_valid_pix = np.sum( ~np.isnan(WQ_array) )
    if (100.0*n_valid_pix/n_water_pix)>=min_valid_pix_thr:
        f_name_noext = "WQ_" + lake_str + "_" + strLS_dates[dateind]
        f_basename = save_dir + f_name_noext
        
        png_fname = f_basename + ".png"
        img.imsave(png_fname, WQ_array)
        
        # Save ancillary data to .json file:
        json_dict = { 'name': lake_str,
                      'displayName': lake_dispname,
                      'date': strLS_dates[dateind],
                      'image': "./" + f_name_noext + ".png",
                      'DateCreated': cur_datetime,
                      'EPSG': WOFS_srs.GetAttrValue("AUTHORITY", 1),
                      'TLcorner': [WOFS_geotx[0], WOFS_geotx[3]],
                      'LRcorner': [WOFS_geotx[0]+WOFS_dy*WOFS_geotx[1], WOFS_geotx[3]+WOFS_dx*WOFS_geotx[5]],
                      'minValue': float( np.nanmin( WQ_array ) ),
                      'maxValue': float( np.nanmax( WQ_array ) ),
                      'meanValue': float( np.nanmean( WQ_array ) ),
                      'medianValue': float( np.nanmedian( WQ_array ) ),
                      'flag': 'undefined',       # this needs to be determined from the data...
                      'lakeType': 'undefined' }  # this should be user-defined somehow...
        
        json_fname = f_basename + ".json"
        with open(json_fname, "w") as outfile:
            json.dump(json_dict, outfile, indent=4)
        
        short_json_dict = { 'name': lake_str,
                            'displayName': lake_dispname,
                            'date': strLS_dates[dateind],
                            'type': WQ_type,
                            'image': "./" + f_name_noext + ".png",
                            'json': "./" + f_name_noext + ".json",
                            'DateCreated': cur_datetime,
                            'flag': 'undefined',
                            'lakeType': 'undefined' }
        contents_json_list.append( short_json_dict )

Write the `contents.json` file...

In [20]:
f_basename = save_dir + "contents.json"
with open(f_basename, "w") as outfile:
    json.dump(contents_json_list, outfile, indent=4)